In [ ]:
## This file is part of Jax Geometry
#
# Copyright (C) 2021, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/jaxgeometry
#
# Jax Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Jax Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Jax Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# LDDMM landmark stochastic dynamics

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.manifolds.landmarks import *
M = landmarks(3)
print(M)
from src.plotting import *

In [ ]:
# Riemannian structure
from src.Riemannian import metric
metric.initialize(M)

## Brownian Motion

In [ ]:
# coordinate form
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

q = M.coords(jnp.vstack((np.linspace(-.5,.5,M.N),np.zeros(M.N))).T.flatten())

_dts = dts(n_steps=1000)
(ts,xs,charts) = M.Brownian_coords(q,_dts,dWs(M.dim,_dts))

# plot
M.newfig()
M.plot()
M.plotx(q,color='r')
M.plot_path(zip(xs,charts))
plt.show()

# plot multiple sample paths
N = 5
xss = np.zeros((N,xs.shape[0],M.dim))
chartss = np.zeros((N,xs.shape[0],q[1].shape[0]))
for i in range(N):
    (ts,xs,charts) = M.Brownian_coords(q,_dts,dWs(M.dim,_dts))
    xss[i] = xs
    chartss[i] = charts

# plot
M.newfig()
M.plot()
colormap = plt.get_cmap('winter')
colors=[colormap(k) for k in np.linspace(0, 1, N)]
for i in range(N):
    M.plot_path(zip(xss[i],chartss[i]),color=colors[i])
M.plotx(q,color='r')
plt.show()

# Langevin equations
see https://arxiv.org/abs/1605.09276

In [ ]:
from src.stochastics import Langevin
Langevin.initialize(M)

M.setN(4)
q = M.coords(jnp.vstack((np.linspace(-.5,.5,M.N),np.zeros(M.N))).T.flatten())
v = jnp.array(jnp.vstack((np.zeros(M.N),np.ones(M.N))).T.flatten())

p = M.flat(q,v)
print("q = ", q)
print("p = ", p)

_dts = dts(n_steps=1000)
(ts,qps,charts) = M.Langevin(q,p,.5,.25,_dts,dWs(M.dim,_dts))

# plot
M.newfig()
M.plot()
M.plot_path(zip(qps[:,0,:],charts))
plt.show()

## Stochastic EPDiff / Eulerian

In [ ]:
minx = -2; maxx = 2
miny = -2; maxy = 2

sigmas_x = np.array([[-.5,0.],[-.5,0.],[.5,0.],[.5,0.]])
sigmas_a = np.array([[1.,0.],[0.,1.],[1.,0.],[0.,1.]])

# noise kernels
k_alpha = .5
k_sigma = .5*jnp.diag(jnp.ones(M.m))
    
J = sigmas_x.shape[0]  
print(k_alpha,k_sigma)
inv_k_sigma = jnp.linalg.inv(k_sigma)
k = lambda x: k_alpha*jnp.exp(-.5*jnp.square(jnp.tensordot(x,inv_k_sigma,(x.ndim-1,1))).sum(x.ndim-1))
k_q = lambda q1,q2: k(q1.reshape((-1,M.m))[:,np.newaxis,:]-q2.reshape((-1,M.m))[np.newaxis,:,:])
sigmas = lambda x: jnp.einsum('ij,jd->ijd',k_q(x,sigmas_x),sigmas_a)

# plot all fields
pts = 20
x,y = np.meshgrid(np.linspace(minx,maxx,pts),np.linspace(miny,maxy,pts))
x = x.flatten(); y = y.flatten()
xy = jnp.vstack((x,y)).T

# compute values
sigmasxy = sigmas(xy)

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython import display

# Turn off matplotlib plot in Notebook
plt.ioff()

fig, ax = plt.subplots()

ax.set_xlim(minx,maxx)
ax.set_ylim(miny,maxy)


def animate(i):
    ax.clear()
    return ax.quiver(x,y,sigmasxy[:,i,0],sigmasxy[:,i,1],angles='xy', scale_units='xy', scale=1)

anim = FuncAnimation(fig, animate, frames=J, interval=500, repeat=False)

video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()

In [ ]:
# initialize with specified noise kernel
from src.stochastics import Eulerian
Eulerian.initialize(M,k=k)

q = M.coords(jnp.vstack((np.linspace(-.5,.5,M.N),np.zeros(M.N))).T.flatten())
v = jnp.array(jnp.vstack((np.zeros(M.N),np.ones(M.N))).T.flatten())
p = M.flat(q,v)

# integrate
_dts = dts(n_steps=1000)
(ts,qps,charts) = M.Eulerian(q,p,sigmas_x,sigmas_a,_dts,dWs(J,_dts))

# plot
M.newfig()
M.plot()
M.plot_path(zip(qps[:,0,:],charts))
plt.plot(sigmas_x[:,0],sigmas_x[:,1],'.',color='g',markersize=15)
plt.show()

# Most probable paths for Arnaudon-Cruzeiro model

In [ ]:
# define domain manifold
from src.manifolds.Euclidean import *
N = Euclidean(2)

# define flow field
u = lambda x,qp: jnp.dot(M.K(x[0],qp[0,:]),qp[1,:])

# MPP Kunita equations
from src.dynamics import MPP_Kunita
MPP_Kunita.initialize(M,N,sigmas,u)

# Curvature
from src.Riemannian import curvature
curvature.initialize(N)

print(N)
print(N.g((jnp.zeros(N.dim),N.chart())))

In [ ]:
# Hamiltonian dynamics
from src.dynamics import Hamiltonian
Hamiltonian.initialize(M)

# flow arbitrary points of N
def ode_Hamiltonian_advect(c,y):
    t,x,chart = c
    qp, = y
    q = qp[0]
    p = qp[1]
        
    dxt = jnp.tensordot(M.K(x,q),p,(1,0)).reshape((-1,M.m))
    return dxt

M.Hamiltonian_advect = jit(lambda xs,qps,dts: integrate(ode_Hamiltonian_advect,None,
                                                        xs[0].reshape((-1,M.m)),xs[1],dts,qps))

In [ ]:
# landmark flow
_dts = dts(n_steps=100)
(_,qps,charts_qp) = M.Hamiltonian_dynamics(q,p,_dts)
dqps = jnp.einsum('t...,t->t...',jnp.gradient(qps,axis=0),1/_dts)

# number of evaluation points
K = 50
x0s = jnp.vstack((np.linspace(minx+.3,maxx-.3,K),np.zeros(K)-.5)).T
_,xs,_ = M.Hamiltonian_advect((x0s.flatten(),M.chart()),qps,_dts)

N.newfig()
N.plot()
plt.plot(sigmas_x[:,0],sigmas_x[:,1],'.',color='g',markersize=15)
for i in range(K):
    x0 = x0s[i]
    v0 = jnp.tensordot(M.K(x0,qps[0,0]),qps[0,1],(1,0))
    (_,xx1,charts) = M.MPP_AC((x0,N.chart()),v0,qps,dqps,_dts)
#     (_,xx1,charts) = M.MPP_AC((q[0][i*N.dim:(i+1)*N.dim],N.chart()),v[i*N.dim:(i+1)*N.dim],qps,dqps,_dts)

    N.plot_path(zip(xx1[:,0,:],charts))
    N.plot_path(zip(xs[:,i],charts),color='r')
#     N.plot_path(zip(qps[:,0,i*N.dim:(i+1)*N.dim],charts_qp),color='r')

plt.show()

# High-dimensional Eulerian systems

In [ ]:
from scipy import misc
import jax.scipy as jsp

# generate stochastic images
global key
keys = jax.random.split(key)
key = keys[0]
subkeys = keys[1:]
%time images = jnp.sqrt(T/n_steps)*random.normal(subkeys[0],(n_steps,64,64,M.m))
image = images[0,:,:]

print("Size of noise basis: ", images.shape[1]*images.shape[2])

# Smooth the noisy image with a 2D Gaussian smoothing kernel.
scale = 1
x = jnp.linspace(-3, 3, 17)
window = jsp.stats.norm.pdf(x,0,scale) * jsp.stats.norm.pdf(x[:, None],0,scale)
%time smooth_image = jax.vmap(lambda im: jsp.signal.convolve(im, window, mode='same'),2,2)(image)

# plot
fig, ax = plt.subplots(M.m, 2, figsize=(12, 10))
for i in range(M.m):
    ax[i,0].imshow(image[:,:,i], cmap='binary_r')
    ax[i,0].set_title('original')
    ax[i,1].imshow(smooth_image[:,:,i], cmap='binary_r')
    ax[i,1].set_title('convolved');


In [ ]:
X, Y = np.meshgrid(jnp.linspace(0,63,35),jnp.linspace(0,63,35))
new_image = jax.vmap(lambda im: jsp.ndimage.map_coordinates(im.T,jnp.vstack((X.flatten(),Y.flatten())),order=1),
                     2,1)(smooth_image).reshape(X.shape+(M.m,))
fig, ax = plt.subplots(M.m, 2, figsize=(12, 10))
for i in range(M.m):
    ax[i,0].imshow(smooth_image[:,:,i], cmap='binary_r')
    ax[i,0].set_title('convolvedl')
    ax[i,1].imshow(new_image[:,:,i], cmap='binary_r')
    ax[i,1].set_title('interploated');

In [ ]:
from matplotlib import cm
(_,ax) = newfig3d()
X, Y = np.meshgrid(x,x)
surf = ax.plot_surface(X,Y,window,cmap=cm.coolwarm)
# Customize the z axis.
ax.set_zlim(0, .2)
fig.colorbar(surf, shrink=0.5, aspect=5, ax=ax)
plt.show()

In [ ]:
# smoothing kernel for Q^{1/2}
kernel_dim = 17
x = jnp.linspace(-3, 3, kernel_dim)
window = jsp.stats.norm.pdf(x) * jsp.stats.norm.pdf(x[:, None],0)
convolve = jax.vmap(lambda dW: jsp.signal.convolve(dW, window, mode='same'),2,2)
interpolate = jax.vmap(lambda Q12dW,q: jsp.ndimage.map_coordinates(Q12dW.T,q.T,order=1),(2,None),1)

def sde_Eulerian_infdim_noise(q,dW):
    # multiply noise on Q^{1/2}
    sqrtQdW = convolve(dW)
    # evluate at x
    dx = interpolate(sqrtQdW,q.reshape((M.N,M.m)))
    return dx

# generate noise for all t
global key
keys = jax.random.split(key)
key = keys[0]
subkeys = keys[1:]
%time dW = jnp.sqrt(T/n_steps)*random.normal(subkeys[0],(n_steps,64,64,M.m))

# evaluate sde function on position q and noise dW
sde_Eulerian_infdim_noise(q[0],dW[0])
%time sde_Eulerian_infdim_noise(q[0],dW[0])
None

In [ ]:
M.N = 64
phis = jnp.linspace(0,2*jnp.pi,M.N)
q = M.coords(jnp.vstack((jnp.cos(phis),jnp.sin(phis))).T.flatten())

# # plot
# M.newfig()
# M.plot()
# M.plotx(q)
# plt.show()

In [ ]:
# generate noise for all t
global key
keys = jax.random.split(key)
key = keys[0]
subkeys = keys[1:]
dW = jnp.sqrt(T/n_steps)*random.normal(subkeys[0],(n_steps,64,64,M.m))

coords_to_pixels = lambda q: (24*q.reshape((-1,M.m))+jnp.array([32,32])[np.newaxis,:])

# smoothing kernel for Q^{1/2}
kernel_dim = 17
x = jnp.linspace(-3, 3, kernel_dim)
window = lambda amp,scale: amp*scale*jnp.sqrt(2*jnp.pi)*jsp.stats.norm.pdf(x,0,scale) * jsp.stats.norm.pdf(x[:, None],0,scale)
# convolve = jax.vmap(lambda amp,scale,dW: jsp.signal.convolve(dW, window(amp,scale), mode='same'),(None,None,2),2)
convolve = jax.vmap(jax.vmap(lambda amp,scale,dW: jsp.signal.convolve(dW, window(amp,scale), mode='same'),(None,None,2),2),(None,None,0),0)
interpolate = jax.vmap(lambda Q12dW,q: jsp.ndimage.map_coordinates(Q12dW.T,coords_to_pixels(q).T,order=1),(2,None),1)
    
def sde_Eulerian_infdim_noise(c,y):
    t,q,_ = c
    dt,sqrtQdW = y

    X = None # to be implemented
    det = jnp.zeros_like(q)
    # evaluate at x
    sto = interpolate(sqrtQdW,q.reshape((M.N,M.m))).flatten()
    return (det,sto,X)

Eulerian_q = lambda q,dts,dW: integrate_sde(sde_Eulerian_infdim_noise,integrator_ito,None,q[0],q[1],dts,dW)
Eulerian = lambda q,amp,scale,dts,dW: Eulerian_q(q,dts,convolve(amp,scale,dW))[0:3]
    
# integrate
amp = .1
scale = 10
Eulerian(q,amp,scale,dts(),dW)
%time (ts,qs,charts) = Eulerian(q,amp,scale,dts(),dW)
print(qs.shape)

# # plot
# M.newfig()
# M.plot()
# M.plot_path(zip(qs,charts))
# plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython import display

# Turn off matplotlib plot in Notebook
plt.ioff()

fig, ax = plt.subplots()

line, = ax.plot([],'*')

ax.set_xlim(-1.2, 1.2)
ax.set_ylim(-1.2, 1.2)


def animate(i):
    line.set_data((qs[i].reshape((-1,M.m))[:,0],qs[i].reshape((-1,M.m))[:,1]))
    return line

# animate(0)
# plt.show()

anim = FuncAnimation(fig, animate, frames=n_steps, interval=100)

video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()

In [ ]:
# # checks
# N.newfig()
# N.plot()
# qs = qps[:,0,:].reshape((-1,M.N,M.m))
# qsint = (qps[0][jnp.newaxis]+jnp.cumsum(np.einsum('t...,t->t...',dqps,_dts),axis=0))[:,0,:].reshape((-1,M.N,M.m))
# for i in range(M.N):
#     plt.plot(qs[:,i,0],qs[:,i,1])
#     plt.plot(qsint[:,i,0],qsint[:,i,1],'*')    
# plt.show()

# N.newfig()
# N.plot()

# qs = qps[:,0,:].reshape((-1,M.N,M.m))
# qsint = (qps[0][jnp.newaxis]+jnp.cumsum(np.einsum('t...,t->t...',dqps,_dts),axis=0))[:,0,:].reshape((-1,M.N,M.m))
# for i in range(M.N):
#     (_,xx1,charts) = M.MPP_AC((q[0][i*N.dim:(i+1)*N.dim],N.chart()),v[i*N.dim:(i+1)*N.dim],qps,dqps,_dts)
#     xs = xx1[:,0]
#     x1s = xx1[:,1]
#     dxs = jnp.einsum('t...,t->t...',jnp.gradient(xs,axis=0),1/_dts)
    
#     xsint1 = xs[0][jnp.newaxis]+jnp.cumsum(np.einsum('t...,t->t...',dxs,_dts),axis=0)
#     xsint2 = xs[0][jnp.newaxis]+jnp.cumsum(np.einsum('t...,t->t...',x1s,_dts),axis=0)
    
#     plt.plot(xs[:,0],xs[:,1])
#     plt.plot(xsint1[:,0],xsint1[:,1],'x')    
#     plt.plot(xsint2[:,0],xsint2[:,1],'*')    
# plt.show()